In [ ]:
import extractTITMC as titmc
from tqdm.notebook import tqdm
import zipfile
import glob
import os
import pprint
pp = pprint.PrettyPrinter(indent=4, depth=6)
os.getcwd()

In [ ]:
%load_ext autoreload
%autoreload 2

# Unzip XML and run code

Double checker avec le moteur de recherche INPI: https://data.inpi.fr/

Le path des data:

`C:\Users\PERNETTH\Documents\Projects\Data\INPI\All\TITMC\Raw\Flux`

Les flux sont composés d'années, de mois et de jours.

Il faut iterer sur plusieurs sous dossier.

Steps:

- Créer une liste avec l'ensemble des paths contenants les XLM

## Attention 

Les fichiers de flux en provenance des greffes sont transmis à une fréquence quotidienne, en fonction de l’actualité des greffes.
Il est possible que certains jours, certains greffes n’envoient pas de données, ou bien que certains greffes envoient plusieurs flux de données par jour.
Les fichiers de flux sont répartis dans 2 sous répertoires en fonction de leur provenance :

- Tribunaux d’instance et mixtes de commerce : répertoire public > IMR_Données_Saisies > titmc > flux

A l’intérieur de ces répertoires, les fichiers sont organisés par date de transmission, puis par greffe, puis par numéro de transmission. Ils sont accompagnés de fichiers md5.

Exemple:

* 20180531220508TITMCFLUX ->20180531 2205 08
  *  date de transmission: 20180531
  * greffe:
  * numéro de transmission:

In [ ]:
import re

In [ ]:
path = r"C:\Users\PERNETTH\Documents\Projects\GitLab\CreationDataBase\RawData\titmc\flux"
list_zip = []

reg2017 = '|2019'.join(['201701', '02',
                        '03', '04',
                        '05', '06', '07',
                        '08', '09', '10',
                        '11', '12'])
regex2018 = '|2018'.join(['201806', '07', '08', '09', '10', '11', '12'])
regex = '{}|{}'.format(reg2017, regex2018)

for root, dirs, files in os.walk(path):
    # Test-> on veut pas avant mai 2018
    for name in files:
        if name.endswith((".zip")):
            matches = re.search(regex, name)
            if matches:
                path_xml = "{}\{}".format(
                    root,
                    name)
                list_zip.append(path_xml)
            else:
                pass

In [ ]:
len(list_zip)

In [ ]:
list_zip[3]

### Toutes les valeurs

In [ ]:
batch_ = []
#path = r"C:\Users\PERNETTH\Documents\Projects\Data\INPI\All\TITMC\Raw\Stock\05"

for file in tqdm(list_zip):
    zipfilePath = (file)
    zip = zipfile.ZipFile(zipfilePath)
    zip.extractall(".")
    zip.close()
    filename = os.path.basename(file).split('.')[0]
    filename_xml = "{}.xml".format(filename)
    data = titmc.getTITMCData(filename_xml)
    batch_.extend(data)
    os.remove(filename_xml)

In [ ]:
len(batch_)

In [ ]:
for i, value in enumerate(data):
    if value["SIREN"] == '852372861':
        pp.pprint(data[i])

### Save to json

L'ensemble des stocks font 550mo

In [ ]:
with open('TITMC_flux.json', 'w') as fp:
    json.dump(batch_, fp)